In [1]:
# Library Imports.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns

# Allows plots to appear directly in the notebook.
%matplotlib inline

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score 

import pickle

In [2]:
# Read CSV file into Data Frame:
trips = pd.read_csv('v3b_tripsCombine_LT_2018.csv', keep_default_na=True, delimiter=',', skipinitialspace=True)

In [3]:
# Read leave1Combine CSV file into Data Frame:
leave1 = pd.read_csv('v3a_leave1Combine_LT_2018.csv', keep_default_na=True, delimiter=',', skipinitialspace=True)

In [4]:
trips.head()

,trip_leave_id,LINEID,num_lineID,DIRECTION,actual_duration,dayOfWeek,weekend,rushHour
0,201802076253783,68,106,1,2924,2,0,0
1,201802076262138,25B,39,2,6292,2,0,0
2,201802076254942,45A,85,2,4247,2,0,0
3,201802076259460,25A,38,1,4020,2,0,0
4,201802076253175,14,13,1,3074,2,0,0


In [5]:
leave1.head()

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,year,month,day,id,trip_leave_id
0,2018-01-01,5955277,1,7347,30000,30000,30000,30000,1001127,2018,1,1,20180101595527701,201801015955277
1,2018-01-01,5955277,2,3669,30080,30080,30080,30080,1001127,2018,1,1,20180101595527702,201801015955277
2,2018-01-01,5955277,3,7349,30138,30138,30138,30138,1001127,2018,1,1,20180101595527703,201801015955277
3,2018-01-01,5955277,4,1631,30206,30206,30206,30206,1001127,2018,1,1,20180101595527704,201801015955277
4,2018-01-01,5955277,5,1632,30221,30221,30221,30221,1001127,2018,1,1,20180101595527705,201801015955277


In [6]:
trips.dtypes

trip_leave_id       int64
LINEID             object
num_lineID          int64
DIRECTION           int64
actual_duration     int64
dayOfWeek           int64
weekend             int64
rushHour            int64
dtype: object

In [7]:
leave1.dtypes

DAYOFSERVICE       object
TRIPID              int64
PROGRNUMBER         int64
STOPPOINTID         int64
PLANNEDTIME_ARR     int64
PLANNEDTIME_DEP     int64
ACTUALTIME_ARR      int64
ACTUALTIME_DEP      int64
VEHICLEID           int64
year                int64
month               int64
day                 int64
id                  int64
trip_leave_id       int64
dtype: object

## Combine Trip & Leave Data

In [8]:
tripLeave1 = pd.merge(leave1, trips, how="left", on=["trip_leave_id"])

In [9]:
tripLeave1.head()

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,year,...,day,id,trip_leave_id,LINEID,num_lineID,DIRECTION,actual_duration,dayOfWeek,weekend,rushHour
0,2018-01-01,5955277,1,7347,30000,30000,30000,30000,1001127,2018,...,1,20180101595527701,201801015955277,16,24.0,1.0,3155.0,0.0,0.0,1.0
1,2018-01-01,5955277,2,3669,30080,30080,30080,30080,1001127,2018,...,1,20180101595527702,201801015955277,16,24.0,1.0,3155.0,0.0,0.0,1.0
2,2018-01-01,5955277,3,7349,30138,30138,30138,30138,1001127,2018,...,1,20180101595527703,201801015955277,16,24.0,1.0,3155.0,0.0,0.0,1.0
3,2018-01-01,5955277,4,1631,30206,30206,30206,30206,1001127,2018,...,1,20180101595527704,201801015955277,16,24.0,1.0,3155.0,0.0,0.0,1.0
4,2018-01-01,5955277,5,1632,30221,30221,30221,30221,1001127,2018,...,1,20180101595527705,201801015955277,16,24.0,1.0,3155.0,0.0,0.0,1.0


In [10]:
tripLeave1.dtypes

DAYOFSERVICE        object
TRIPID               int64
PROGRNUMBER          int64
STOPPOINTID          int64
PLANNEDTIME_ARR      int64
PLANNEDTIME_DEP      int64
ACTUALTIME_ARR       int64
ACTUALTIME_DEP       int64
VEHICLEID            int64
year                 int64
month                int64
day                  int64
id                   int64
trip_leave_id        int64
LINEID              object
num_lineID         float64
DIRECTION          float64
actual_duration    float64
dayOfWeek          float64
weekend            float64
rushHour           float64
dtype: object

In [11]:
tripLeave1.isnull().sum()

DAYOFSERVICE             0
TRIPID                   0
PROGRNUMBER              0
STOPPOINTID              0
PLANNEDTIME_ARR          0
PLANNEDTIME_DEP          0
ACTUALTIME_ARR           0
ACTUALTIME_DEP           0
VEHICLEID                0
year                     0
month                    0
day                      0
id                       0
trip_leave_id            0
LINEID             1030738
num_lineID         1030738
DIRECTION          1030738
actual_duration    1030738
dayOfWeek          1030738
weekend            1030738
rushHour           1030738
dtype: int64

## Create a New Dataframe for the NaNs:

In [12]:
tripLeave1_NaNs = tripLeave1[(tripLeave1['LINEID'].isnull())]

In [14]:
tripLeave1_NaNs.head()

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,year,...,day,id,trip_leave_id,LINEID,num_lineID,DIRECTION,actual_duration,dayOfWeek,weekend,rushHour
1394,2018-01-01,5955353,1,4620,82920,82920,82920,82920,2534831,2018,...,1,20180101595535301,201801015955353,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1395,2018-01-01,5955353,2,7563,82955,82955,82955,82955,2534831,2018,...,1,20180101595535302,201801015955353,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1396,2018-01-01,5955353,3,3368,82992,82992,82992,82992,2534831,2018,...,1,20180101595535303,201801015955353,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1397,2018-01-01,5955353,4,4626,83058,83058,83058,83058,2534831,2018,...,1,20180101595535304,201801015955353,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1398,2018-01-01,5955353,5,4627,83093,83093,83093,83093,2534831,2018,...,1,20180101595535305,201801015955353,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
tripLeave1_NaNs.shape

(1030738, 21)

In [15]:
# Drop NaN rows from tripLeave1
tripLeave1.drop(tripLeave1[tripLeave1['LINEID'].isnull()].index, inplace = True)

In [16]:
tripLeave1.isnull().sum()

DAYOFSERVICE       0
TRIPID             0
PROGRNUMBER        0
STOPPOINTID        0
PLANNEDTIME_ARR    0
PLANNEDTIME_DEP    0
ACTUALTIME_ARR     0
ACTUALTIME_DEP     0
VEHICLEID          0
year               0
month              0
day                0
id                 0
trip_leave_id      0
LINEID             0
num_lineID         0
DIRECTION          0
actual_duration    0
dayOfWeek          0
weekend            0
rushHour           0
dtype: int64

In [18]:
tripLeave1.shape

(18362590, 21)

## Send the tripLeave1_NaNs Dataframe to CSV

In [19]:
# Send tripLeave1_NaNs to csv:
tripLeave1_NaNs.to_csv('v4_tripLeave1_NaNs_LT_2018.csv', index=False)

## Send tripLeave1 to CSV

In [31]:
# Send tripLeave1 to csv:
tripLeave1.to_csv('v4_tripLeave1_LT_2018.csv', index=False)